# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os

import azureml.core
import numpy as np
import pandas as pd
import pkg_resources
from pathlib import Path
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.datastore import Datastore
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-capstone-expirement'

experiment=Experiment(ws, experiment_name)

In [3]:
found = False
key = "udacity-stroke-dataset"
description_text = "Stroke Dataset for Udacity Capstone"

dataset = None

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 


if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/ciccioska/Udacity-AzureML-Capstone-Project/master/healthcare-dataset-stroke-data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data) 
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)



x_df = dataset.to_pandas_dataframe().dropna()

x_df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,1.000000


In [4]:
# Normalize
x_df["work_type"] = x_df.work_type.apply(lambda s: 1 if s == "Private" else 0 if s == "Self-employed" else -1)
x_df["bmi"] = x_df.bmi.apply(lambda s: 0 if s == "N/A" else s)
x_df["gender"] = x_df.gender.apply(lambda s: 1 if s == "Male" else 0)
x_df["smoking_status"] = x_df.smoking_status.apply(lambda s: 1 if s == "smokes" else -1 if s == "never smoked" else 2 if s == "formerly smoked" else 0)
x_df["Residence_type"] = x_df.Residence_type.apply(lambda s: 1 if s == "Urban" else 0)

y_df = x_df.pop("stroke")

# Split
(x_train, x_test, y_train, y_test) = train_test_split(x_df, y_df, test_size = 0.3)

train_data_df = pd.concat([x_train, y_train], axis=1)
test_data_df = pd.concat([x_test, y_train], axis=1)

# view train dataset
train_data_df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
3836,71396,1,3.0,0,0,False,-1,1,105.34,15.5,0,0
4688,40041,1,31.0,0,0,False,0,0,64.85,23,0,0
4258,22330,0,45.0,0,0,True,0,1,82.94,29.3,0,0
2391,35974,0,16.0,0,0,False,1,0,86.32,18.3,0,0
2172,3842,1,73.0,0,0,True,1,0,86.57,28.5,2,0


In [5]:
# save our train data
Path("data").mkdir(parents=True, exist_ok=True)
    
pd.DataFrame(train_data_df).to_csv("data/dataset.csv", index=False)

# upload
datastore = ws.get_default_datastore()
datastore.upload(src_dir='./data', target_path='classified', overwrite=True, show_progress=True)

 # datastore for AutoML
training_data = TabularDatasetFactory.from_delimited_files(path=datastore.path('classified/dataset.csv'))

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


In [6]:
cluster_name = "udacity-capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
automl_settings = {
    "n_cross_validations": 2,
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=training_data,
                             label_column_name="stroke",   
                             path = './udacity-capstone-project',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-capstone-expirement,AutoML_b8d6e86d-f7c6-46ef-85e7-ec2517c9f183,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_b8d6e86d-f7c6-46ef-85e7-ec2517c9f183',
 'target': 'udacity-capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-23T07:59:11.754089Z',
 'endTimeUtc': '2022-09-23T08:26:19.54181Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'udacity-capstone-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-capstone-expirement","subscription_id":"ac113154-2e6f-499c-9be7-09bd5fba4d3a","resource_group":"udacity-capstone-rg","workspace_name":"udacity-capstone","region":"westeurope","com

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_automl_run, automl_model = remote_run.get_output()

# BestRun
print(best_automl_run)

# Params
print(automl_model.steps[1][1].estimators)

# Accuracy of BestRun
print('Accuracy of BestRun = ')
print(best_automl_run.properties['score'])

Package:azureml-automl-runtime, training version:1.45.0, current version:1.44.0
Package:azureml-core, training version:1.45.0, current version:1.44.0
Package:azureml-dataset-runtime, training version:1.45.0, current version:1.44.0
Package:azureml-defaults, training version:1.45.0, current version:1.44.0
Package:azureml-interpret, training version:1.45.0, current version:1.44.0
Package:azureml-mlflow, training version:1.45.0, current version:1.44.0
Package:azureml-pipeline-core, training version:1.45.0, current version:1.44.0
Package:azureml-responsibleai, training version:1.45.0, current version:1.44.0
Package:azureml-telemetry, training version:1.45.0, current version:1.44.0
Package:azureml-train-automl-client, training version:1.45.0, current version:1.44.0
Package:azureml-train-automl-runtime, training version:1.45.0, current version:1.44.0
Package:azureml-train-core, training version:1.45.0, current version:1.44.0
Package:azureml-train-restclients-hyperdrive, training version:1.45.

Run(Experiment: udacity-capstone-expirement,
Id: AutoML_b8d6e86d-f7c6-46ef-85e7-ec2517c9f183_50,
Type: azureml.scriptrun,
Status: Completed)
[('31', Pipeline(memory=None,
         steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features=0.8,
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=0.01,
                                        min_samples_split=0.19736842105263158,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=400, n_jobs=1,
              

In [12]:
#TODO: Save the best model
automl_model = remote_run.register_model(model_name='stroke-udacity-experiment-model')

automl_model.download(target_dir="outputs", exist_ok=True)

'outputs/model.pkl'

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
